In [ ]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()

dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 25)
dataset, frequency_list = text_preprocessing.keep_labels(preprocessded_dataset,keep_ratio=0.10)
slice_dataset = text_preprocessing.dataset_slice(dataset,ratio=0.25)

import numpy as np
all_sente = list(slice_dataset['text'])
all_label = np.array(slice_dataset.drop('text', 1))

X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sente, all_label)

In [ ]:
from multilab.models import Elmo

config = {
                         'no_of_labels'               : len(y_train[0]),
                         'learning_rate'              : 0.001,
                         'epoch'                      : 2,
                         'batch_size'                 : 128,
                         'result_path'                : '/Users/monk/Desktop'
                        }


el_m = Elmo(X_train, y_train, X_test,  y_test, config)

In [ ]:
print(el_m.train())

In [ ]:
# testing

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub

ins = tf.placeholder(tf.string, (None), name='pl_phrase_text')


module = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)
embeddings = module(dict(text=ins))
embeddings = tf.expand_dims(embeddings, axis=1)

In [ ]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    print(sess.run(embeddings,feed_dict={ins:['hello']}).shape)

In [33]:
import tensorflow as tf 
import tensorflow_hub as hub

tokens = tf.placeholder(tf.string, (None), name='pl_phrase_text')
pl_phrase_len = tf.placeholder(tf.int32, (None), name='pl_phrase_len')

tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]


module = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)
module_features = module(dict(tokens=tokens, sequence_len = pl_phrase_len),
                                 signature='tokens', as_dict=True)
embeddingse = module_features["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [34]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    data,t = sess.run([embeddingse,module_features],feed_dict={tokens: tokens_input, pl_phrase_len: tokens_length})
    
    
    

In [35]:
data.shape

(2, 6, 1024)

In [28]:
data[0].shape

(6, 1024)

In [22]:
data[1]

{'sequence_len': array([6, 5], dtype=int32),
 'elmo': array([[[ 3.08154464e-01,  2.66303837e-01,  2.35613152e-01, ...,
          -3.70856762e-01,  1.64905071e-01, -7.24593699e-02],
         [ 5.14287412e-01, -1.35323137e-01,  1.10904127e-01, ...,
           4.04687077e-02, -4.78973389e-02,  7.36596406e-01],
         [-2.58805454e-02, -7.28362352e-02, -7.93559626e-02, ...,
          -2.90724307e-01,  7.24214077e-01,  4.38635081e-01],
         [-3.47980261e-01, -2.91023925e-02, -8.19930553e-01, ...,
          -9.20483828e-01,  2.18880177e-02,  1.21060461e-01],
         [-2.18274534e-01, -1.30765766e-01, -2.52096236e-01, ...,
          -2.96935469e-01, -1.58281043e-01, -4.90074344e-02],
         [ 1.00725375e-01, -2.95346603e-02, -2.44942963e-01, ...,
          -3.72350782e-01, -1.48758322e-01,  2.15922594e-01]],
 
        [[ 5.45786619e-02, -2.64275581e-01,  4.68437970e-01, ...,
          -1.40771136e-01, -2.65682638e-01,  4.52120036e-01],
         [ 8.09431151e-02,  1.15838900e-01, -1.5

In [19]:
from tqdm import tqdm
import nltk

# padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]


sentences = ['check this out','i am very happy', 'what is', ['check','this'],'he a a a abs']


def pad_sentences(self, sentences, sequence_len):
    padded_sentences = []
    actual_length    = []


    for sentence in tqdm(sentences):
        if not isinstance(sentence, list):
            token = nltk.word_tokenize(sentence)
        else:
            token = sentence
        
        if len(token) < sequene_len:
            actual_length.append(len(token))
            token = token + [''] * (sequene_len-len(token))
        else:
            actual_length.append(len(token))
        
        padded_sentences.append(token)
    return padded_sentences, actual_length

In [20]:
chy = pad_sentences(sentences,5)

100%|██████████| 5/5 [00:00<00:00, 1920.12it/s]


In [21]:
chy

([['check', 'this', 'out', '', ''],
  ['i', 'am', 'very', 'happy', ''],
  ['what', 'is', '', '', ''],
  ['check', 'this', '', '', ''],
  ['he', 'a', 'a', 'a', 'abs']],
 [3, 4, 2, 2, 5])